In [ ]:
# initializing
library(tidyverse)
library(scales)
library(tidymodels)

data <- read_csv("data/players.csv") |>
    mutate(played_hours = played_hours + 0.01) |>
    mutate(subscribe = as_factor(subscribe))

In [1]:
# plotting the relationship between age and played hours
dataplot <- ggplot(data, aes(x = Age, y = played_hours, color = subscribe)) +
    geom_point(alpha= 0.4) +
    scale_y_log10(labels = label_comma())

dataplot

ERROR: Error in ggplot(data, aes(x = Age, y = played_hours, color = subscribe)): could not find function "ggplot"


In [ ]:

set.seed(1)

split <- initial_split(data, prop = 0.75, strata = subscribe)
train <- training(split)
test <- testing(split)

recipe <- recipe(subscribe ~ played_hours, data = data) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

vfold <- vfold_cv(train, v = 5, strata = subscribe)
k_vals <- tibble(neighbors = seq(from = 1, to = 50, by = 1))

knn_fit <- workflow() |>
  add_recipe(recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = vfold, grid = k_vals) |>
  collect_metrics()

k_neighbors_plot <- ggplot(knn_fit, aes(x = neighbors, y = mean)) +
    geom_point()
k_neighbors_plot

knn_fit_neighbors